In [ ]:
import pandas as pd

df_output = pd.read_csv('input_files/translations_vehicle360_sg_v1.0.csv')

df_output["Modified"] = ""
col_modified = df_output['Modified']
df_output.drop(labels=['Modified'], axis=1,inplace = True)
df_output.insert(0, 'Modified', col_modified)

df_input_1 = pd.read_csv('input_files/bi.csv')
df_input_2 = pd.read_csv('input_files/biWebApp.csv')
df_input_3 = pd.read_csv('input_files/inside_ffu_module.csv')
df_input_4 = pd.read_csv('input_files/sales-module.csv')
df_input_5 = pd.read_csv('input_files/vp-modules.csv')

# Concatenation of all the given files 
# (easy because all the columns are the same)
df_master = pd.concat([df_input_1, df_input_2,  df_input_3, df_input_4, df_input_5])

In [ ]:
from rapidfuzz import fuzz, process
import deepl

auth_key = '325c848e-727d-6daf-ec26-cee3d14b11cc:fx'
translator = deepl.Translator(auth_key)

def deepl_translate(word):
    translation_DE = translator.translate_text(word, target_lang="DE", formality="more").text
    translation_FR = translator.translate_text(word, target_lang="FR", formality="more").text
    translation_IT = translator.translate_text(word, target_lang="IT", formality="more").text
    return translation_DE,translation_FR,translation_IT,'Deepl'

def fuzzy_search(word):
    if '{' in word:
        return '','','','Ignored'
    else:
        df_master_filled = df_master['EN'].fillna('')
        most_similar = process.extractOne(word, df_master_filled, scorer=fuzz.token_sort_ratio, score_cutoff = 90)
        if most_similar:
            df_temp = df_master.loc[df_master['EN'] == most_similar[0]]
            return df_temp['DE'].values[0], df_temp['FR'].values[0], df_temp['IT'].values[0], 'Fuzzy: '+most_similar[0]+',  %8.2f' % most_similar[1]
        else:
            return deepl_translate(word)

def find_DE_translations(word):
    df_temp = df_master.loc[df_master['DE'] == word]
    if df_temp.empty:
        return ''
    else:
        return df_temp['EN'].values[0]

def find_EN_translations(word):
    df_temp = df_master.loc[df_master['EN'] == word]
    if df_temp.empty:
        return fuzzy_search(word)
    else:
        return df_temp['DE'].values[0], df_temp['FR'].values[0], df_temp['IT'].values[0], 'Identical match'

def find_FR_translations(word):
    if '{' in word:
        return ''
    else:    
        return translator.translate_text(word, target_lang="FR", formality="more").text

def find_IT_translations(word):
    if '{' in word:
        return ''
    else:  
        return translator.translate_text(word, target_lang="IT", formality="more").text


In [ ]:
df_output_EN_nan = df_output[df_output['EN'].isna()]

for i in df_output_EN_nan.index:
    # Find row with translations in masterfile
    en_value = find_DE_translations(df_output_EN_nan.at[i, 'DE'])
    df_output.at[i, 'EN'] = en_value

df_output_EN_nan

In [ ]:
df_output_DE_nan = df_output[df_output['DE'].isna()]

for i in df_output_DE_nan.index:
    # Find row with translations in masterfile
    cell_DE, cell_FR, cell_IT, cell_mod = find_EN_translations(df_output_DE_nan.at[i, 'EN'])
    df_output.at[i, 'DE'] = cell_DE
    df_output.at[i, 'FR'] = cell_FR
    df_output.at[i, 'IT'] = cell_IT
    df_output.at[i, 'Modified'] = cell_mod

In [ ]:
df_output_FR_nan = df_output[df_output['FR'].isna()]

for i in df_output_FR_nan.index:
    # Find row with translations in masterfile
    cell_FR = find_FR_translations(df_output_FR_nan.at[i, 'EN'])
    df_output.at[i, 'FR'] = cell_FR
    df_output.at[i, 'Modified'] += ' + FR'

In [ ]:
df_output_IT_nan = df_output[df_output['IT'].isna()]

for i in df_output_IT_nan.index:
    # Find row with translations in masterfile
    cell_IT = find_IT_translations(df_output_IT_nan.at[i, 'EN'])
    df_output.at[i, 'IT'] = cell_IT
    df_output.at[i, 'Modified'] += ' + IT'

In [ ]:
df_output

In [ ]:
df_output.to_csv("output_files/translations_vehicle360_sg_v1.0_updated.csv",index=None)

In [ ]:
df_output.iloc[[458]]